In order to perform iCLIP2 data analysis, we would have following files with us.
1. iCLIP2 fastq files (sequencing data)
2. Human genome fasta File 
3. Human genome annotation file (GTF file)
4. Virus genome fasta file
5. Virus genome annotation file (GFF/GTF file)
6. iCLIP2 barcode file

Step 1 is to have all files in our respective folders

1. Fetching sequencing data fastq files from given folder to my analysis folder.
path given was as follows:
"/home5/castello_group/sequencing_data/iClip/240523_NB552751_0021_AHGKCCBGXW_iClip_NOP2_Innes/"

In [3]:
!cd /home2/2459810a/iCLIP_analysis/NOP2_Innes/
!mkdir -p Data/Merged

!cat \
/home5/castello_group/sequencing_data/iClip/240523_NB552751_0021_AHGKCCBGXW_iClip_NOP2_Innes/iClip_NOP2_Innes_S1_R1_001.fastq.gz >\
Data/Merged/NOP2_iCLIP_RAW.fastq.gz 

The above code has given us our sequencing file in /Data/Merged folder.
Sources of data used are:

Human annotation: http://ftp.ensembl.org/pub/release-106/gtf/homo_sapiens/
Homo_sapiens.GRCh38.106.gtf.gz

Human fasta: http://ftp.ensembl.org/pub/release-106/fasta/homo_sapiens/dna/ Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz

SINV Fasta and GFF files have been copied from PURA analysis folder for the time being as Innes does not have that data with him.

Files have been saved in iCLIP_analysis/NOP2_innes/ folder.

We have to merge human and SINV fasta files to create a combined fasta file for analysis.
Later, we will try to merge annotation files as well for both genomes.

In [6]:
!cat /home2/2459810a/iCLIP_analysis/NOP2_Innes/Homo_sapiens.GRCh38.dna.primary_assembly.fa /home2/2459810a/iCLIP_analysis/NOP2_Innes/SINV_fasta.fa | \
gzip > /home2/2459810a/iCLIP_analysis/NOP2_Innes/HS.GRCh38.SINV.fa.gz

We will Convert downloaded gff to gtf.

There are different requirements of the genome for htseq, relative to alignment. A new 'fixed' version of the SINV annotation needed to be made. This might have been a problem specific to this genome annotation so might not be necessary with the new genome annotation.


In [ ]:
# !conda update -n base -c defaults conda
# conda install -c bioconda agat
!agat_convert_sp_gff2gtf.pl \
--gff /home2/2459810a/iCLIP_analysis/NOP2_Innes/SINV_genome_annotation.gff3 \
--gtf_version 3 \
--output /home2/2459810a/iCLIP_analysis/NOP2_Innes/SINV_anno.gtf



In [ ]:
#  i performed this step on my local computer as i could not resolve library packages in R on VS code soomehow 
library(dplyr)
library(GenomicRanges)
library(rtracklayer)
library(tidyverse)
library(Biostrings)

read_tsv("./SINV_anno.gtf", skip = 3, col_names = FALSE) %>%
  mutate(X9 = str_replace(X9, "original_biotype", "gene_biotype")) %>%
  mutate(X9 = case_when(
    str_detect(X9, "UTR") & !str_detect(X9, "gene_biotype") ~ paste0(X9, " gene_biotype \"rna\";"),
    TRUE ~ X9
  )) %>%
  mutate(X9 = case_when(
    str_detect(X9, "gene_biotype") ~ X9,
    TRUE ~ paste0(X9, " gene_biotype \"protein_coding\";")
  )) %>%
  mutate(X9 = str_remove_all(X9, "\"Genbank:.*\" ")) %>%
  write_tsv("./GENOMEDIR/sliding_window/SINV_annotation_fixed.gtf", 
            col_names = FALSE, escape = "none")

In [3]:
# combining human and SINV annotation 
!mkdir -p /home2/2459810a/iCLIP_analysis/NOP2_Innes/GENOMEDIR/sliding_window

!cat \
/home2/2459810a/iCLIP_analysis/NOP2_Innes/Homo_sapiens.GRCh38.106.gtf \
/home2/2459810a/iCLIP_analysis/NOP2_Innes/GENOMEDIR/SINV_annotation_fixed.gtf >\
/home2/2459810a/iCLIP_analysis/NOP2_Innes/GENOMEDIR/sliding_window/HS.GRCh38.SINV_attributefix.gtf


### Add missing gene_name attributes to ensembl gtf
Ensembl GTF and GFF3 have some missing 'gene_name' attributes --> Fix with pygtftk  

In [ ]:
# conda create -n pygtftk python=3.9
# !conda init
!conda activate pygtftk
# Get number of unique gene_id, gene_name, transcript_id, transcript_name (to check #gene_name < gene_id; #transcript_name < transcript_id)
!gtftk count_key_values \
-u -i /home2/2459810a/iCLIP_analysis/NOP2_Innes/GENOMEDIR/sliding_window/HS.GRCh38.SINV_attributefix.gtf \
-k gene_id,gene_name,transcript_id,transcript_name

# Create new gene_name as gene_name|gene_id
!gtf=/home2/2459810a/iCLIP_analysis/NOP2_Innes/GENOMEDIR/sliding_window/HS.GRCh38.SINV_attributefix.gtf
!base=$(basename "$gtf")
!output="/home2/2459810a/iCLIP_analysis/NOP2_Innes/GENOMEDIR/sliding_window/$(echo ${base} |sed -e 's/.gtf/_with_gn.gtf/')"

!gtftk merge_attr \
-i $gtf \
-k gene_name,gene_id \
-d gene_name > $output

# Create new transcript_name as transcript_name|transcript_id
!gtf=/home2/2459810a/iCLIP_analysis/NOP2_Innes/GENOMEDIR/sliding_window/HS.GRCh38.SINV_attributefix_with_gn.gtf
!base=$(basename "$gtf")
!output="/home2/2459810a/iCLIP_analysis/NOP2_Innes/GENOMEDIR/sliding_window/$(echo ${base} |sed -e 's/_with_gn.gtf/_with_gn_tn.gtf/')"

!gtftk merge_attr \
-i $gtf \
-k transcript_name,transcript_id \
-d transcript_name > $output

# Check the number of gene_id/name and transcript_id/name match
!gtftk count_key_values \
-u -i /home2/2459810a/iCLIP_analysis/NOP2_Innes/GENOMEDIR/sliding_window/HS.GRCh38.SINV_attributefix_with_gn_tn.gtf \
-k gene_id,gene_name,transcript_id,transcript_name

!conda deactivate

In [ ]:
### Flatten annotation


file=/home2/2459810a/iCLIP_analysis/NOP2_Innes/GENOMEDIR/sliding_window/HS.GRCh38.SINV_attributefix_with_gn_tn.gtf
base=$(basename $file _with_gn_tn.gtf)

# /home4/2711498i/miniconda3/envs/htseq-clip/bin/htseq-clip annotation \
htseq-clip annotation \
-g $file \
-u gene_id \
-n gene_name \
-t gene_biotype \
--splitExons \
--unsorted \
-o /home2/2459810a/iCLIP_analysis/NOP2_Innes/GENOMEDIR/sliding_window/$(echo ${base}).flattened.annotation.txt.gz



### Create sliding window

# 50/20 - as in Hentze ENO1 paper
htseq-clip createSlidingWindows \
-i /home2/2459810a/iCLIP_analysis/NOP2_Innes/GENOMEDIR/sliding_window/HS.GRCh38.SINV_attributefix.flattened.annotation.txt.gz \
-w 50 \
-s 20 \
-o /home2/2459810a/iCLIP_analysis/NOP2_Innes/GENOMEDIR/sliding_window//HS.GRCh38.SINV_attribute-fix.flattened.w50s20.txt.gz



### Create annotation ID file


htseq-clip mapToId \
-a /home2/2459810a/iCLIP_analysis/NOP2_Innes/GENOMEDIR/sliding_window//HS.GRCh38.SINV_attribute-fix.flattened.w50s20.txt.gz \
-o /home2/2459810a/iCLIP_analysis/NOP2_Innes/GENOMEDIR/sliding_window/HS.GRCh38.SINV_attribute-fix.flattened.w50s20.maptoid.txt.gz


### Calculate chromosome size 


samtools faidx /home2/2459810a/iCLIP_analysis/NOP2_Innes/HS.GRCh38.SINV.fa.gz

cut -f1,2 /home2/2459810a/iCLIP_analysis/NOP2_Innes/HS.GRCh38.SINV.fa.fai > /home2/2459810a/iCLIP_analysis/NOP2_Innes/GENOMEDIR/sizes.genome





up until this step, genome directory preparation steps are completed. Now we will perform quality checks on fastq reads, remove barcodes and terminal sequences and perform alignment. 

In [ ]:

# Now initial analysis

echo "Remove PhiX reads"

/software/bbmap-v38.90/bbduk.sh \
in=Data/Merged/NOP2_iCLIP_RAW.fastq.gz  \
out=Data/Merged/NOP2_iCLIP_rmPhiX.fastq.gz \
ref=/software/bbmap-v38.90/resources/phix174_ill.ref.fa.gz \
k=31 hdist=1 -Xmx16g threads=8 \
stats=Data/Merged/NOP2_iCLIP_rmPhiX.stats.txt

echo "Full dataset sequencing QC"

mkdir -p Data/Merged/FastQC

for file in Data/Merged/*_rmPhiX.fastq.gz
do
echo "Processing file $file"
fastqc $file -o Data/Merged/FastQC
echo ""
done 

mkdir -p Plots/QC

multiqc -f \
Data/Merged/FastQC/*_fastqc.zip \
-o Plots/QC \
-n 01_Sequencing-runs_MultiQC-report


echo "Count barcode frequencies"

zcat Data/Merged/NOP2_iCLIP_rmPhiX.fastq.gz | awk -v umi1_len=5 -v exp_bc_len=6 '{if (FNR%4==2) print substr($1,(umi1_len+1),exp_bc_len)}' | sort | uniq -c | sort -k1,1rn>barcode_freqs

echo "Demultiplex reads"

mkdir -p Trimming/Demultiplex

java -jar /software/je_1.2/je_1.2_bundle.jar demultiplex \
F1=Data/Merged/NOP2_iCLIP_rmPhiX.fastq.gz \
BF=GENOMEDIR/NOP2_barcodes_file.txt \
RCHAR=':' \
O=Trimming/Demultiplex \
UF1=Unassigned_NOP2_iCLIP_merged_jemultiplexer.fastq \
M=NOP2_iCLIP_merged_jemultiplexer_out_stats.txt \
FASTQ_FILE_EXTENSION=fastq

echo "Demultiplexed reads QC"

mkdir -p Trimming/Demultiplex/FastQC

for file in Trimming/Demultiplex/*.fastq.gz
do
echo "Processing file $file"
fastqc $file -o Trimming/Demultiplex/FastQC
echo ""
done 

multiqc -f \
Trimming/Demultiplex/FastQC/*_fastqc.zip \
-o Plots/QC \
-n 02_Libraries_MultiQC-report

echo "Adapter trimming"

mkdir -p Trimming/Adapter

for file in Trimming/Demultiplex/*.fastq.gz
do
[[ $file == *Unassigned*.fastq.gz ]] && continue 
echo "" 
echo "Processing file $file" 
base=$(basename "$file") 
outfile="$(echo ${base} |sed -e 's/.fastq.gz/_trimmed.fastq.gz/')" 
outshort="$(echo ${base} |sed -e 's/.fastq.gz/_trimmed.fastq.tooshort.gz/')" 
outinfo="$(echo ${base} |sed -e 's/.fastq.gz/_trimmed.info/')" 
cutadapt $file \
-a AGATCGGAAGAGCGGTTCAG \
-j 4 -e 0.1 -O 1 --nextseq-trim 10 --minimum-length 15 \
-o Trimming/Adapter/$outfile \
--too-short-output Trimming/Adapter/$outshort > \
Trimming/Adapter/$outinfo 
echo "" 
done

echo "Adapter trimming QC"

multiqc -f \
Trimming/Adapter/*_trimmed.info \
-o Plots/QC \
-n 03_Cutadapt_MultiQC-report

echo "Assess rRNA contamination"

mkdir -p Trimming/rRNA

for file in Trimming/Adapter/*.fastq.gz
do
echo ""
base=$(basename $file .fastq.gz)
echo "Processing $file"
/software/bbmap-v38.90/bbduk.sh \
-Xmx24G \
in=$file \
out=Trimming/rRNA/$(echo ${base}).minus.rRNA.fastq.gz \
ref=GENOMEDIR/Hsap_rDNA_U13369.1.fa \
k=25 \
stats=Trimming/rRNA/$(echo ${base}).rRNA.stat.txt \
hdist=1
echo ""
done

echo "Unzip fastqs"


for file in Trimming/Adapter/*_trimmed.fastq.gz 
do 
gunzip -f $file 
done


Alignment of the reads to the merged genome of human and SINV

In [ ]:
# make the STAR object

STAR --runThreadN 8 --runMode genomeGenerate \
--genomeFastaFiles HS.GRCh38.SINV.fa \
--sjdbGTFfile GENOMEDIR/sliding_window/HS.GRCh38.SINV_attributefix_with_gn_tn.gtf \
--genomeDir STAR \
--alignSJoverhangMin 8 --sjdbOverhang 100 \
--limitGenomeGenerateRAM 31000000000 --genomeSAindexNbases 10 --genomeSAsparseD 4


In [ ]:
echo "Align reads"

mkdir -p Alignment 

for file in Trimming/Adapter/*_trimmed.fastq 
do 
echo "" 
echo "Processing file $file" 
base=$(basename "$file") 
outdir="$(echo ${base} |sed -e 's/_trimmed.fastq//')" 
mkdir Alignment/$outdir 
STAR --runMode alignReads --runThreadN 8 \
--outSAMtype BAM SortedByCoordinate \
--genomeDir STAR \
--sjdbGTFfile GENOMEDIR/sliding_window/HS.GRCh38.SINV_attributefix_with_gn_tn.gtf \
--readFilesIn $file --outFileNamePrefix Alignment/$outdir/ \
--outReadsUnmapped Fastx \
--outFilterMismatchNmax 999 \
--outFilterMultimapNmax 1 \
--outFilterMismatchNoverLmax 0.04 \
--outSJfilterReads Unique \
--alignEndsType EndToEnd
echo "" 
done 

In [ ]:
multiqc -f \
Alignment/*/*Log.final.out \
-o ./ \
-n 05_STAR_MultiQC-report


mkdir -p Alignment_idxstats

for dir in $(ls -d Alignment/*/) 
do
echo "" 
file=$dir/Aligned.sortedByCoord.out.bam 
base=$(basename "$dir") 
output="Alignment_idxstats/$(echo ${base}).idxstats.txt"  
echo "Processing file $file" 
samtools index $file 
samtools idxstats $file > $output 
done

echo "Alignment QC"

multiqc -f \
Alignment_idxstats/*idxstats.txt \
-o ./ \
-n 06_Chromosome-mapping-pre-dedup_MultiQC-report

echo "Alignment QC"

mkdir -p Dedup 

for dir in $(ls -d Alignment/*/) 
do
echo "" 
input=$dir/Aligned.sortedByCoord.out.bam 
base=$(basename "$dir") 
output="Dedup/$(echo ${base}).dedup.je.bam" 
outlog="Dedup/$(echo ${base}).dedup.je.log" 
metrics="Dedup/$(echo ${base}).dedup.je.metrics" 
java -jar /software/je_1.2/je_1.2_bundle.jar markdupes \
I=$input \
O=$output \
M=$metrics \
REMOVE_DUPLICATES=True \
MM=1 >\
$outlog
echo "" 
done


mkdir -p Dedup_idxstats

for file in $(ls Dedup/*.dedup.je.bam) 
do
echo "" 
echo "Processing file $file" 
base=$(basename "$file") 
output="Dedup_idxstats/$(echo ${base} |sed -e 's/.dedup.je.bam/.idxstats.txt/')" 
samtools index "$file" 
samtools idxstats $file > $output
done


multiqc -f \
Dedup_idxstats/*idxstats.txt \
-o ./ \
-n 07_Chromosome-mapping-post-dedup_MultiQC-report


outfile='Trimming/Demultiplex/Raw-read-counts.txt' 

for file in $(ls Trimming/Demultiplex/*.fastq.gz)
do
echo "File is $file"
base=$(basename "$file" .fastq.gz) 
count=$(echo $(zcat $file | wc -l) / 4 | bc) 
echo "Count is $count"
echo "$base $count">> $outfile
done

outfile='Trimming/Adapter/Trimmed-read-counts.txt' 

for file in $(ls Trimming/Adapter/*.fastq)
do
echo "File is $file"
base=$(basename "$file" _trimmed.fastq) 
count=$(echo $(expr $(cat $file | wc -l) / 4)) 
echo "Count is $count"
echo "$base $count">> $outfile
done


outfile='Alignment/Unique-map-read-counts.txt'

for dir in $(ls -d Alignment/*/) 
do
echo "" 
file=$dir/Aligned.sortedByCoord.out.bam 
base=$(basename "$dir") 
echo "File is $file" 
count=$(samtools view -c $file) 
echo "Count is $count"
echo "$base $count" >> $outfile
done


outfile='Dedup/Dedup-read-counts.txt'

for file in $(ls Dedup/*.dedup.je.bam) 
do
echo "" 
echo "File is $file" 
base=$(basename "$file" .dedup.je.bam) 
count=$(samtools view -c $file) 
echo "Count is $count"
echo "$base $count" >> $outfile
done



In [ ]:
mkdir -p Xlsite/collapsed
mkdir -p Xlsite/shifted/bed

for file in Dedup/*.dedup.je.bam 
do
echo "Processing $file" 
base=$(basename $file) 
outbed="Xlsite/collapsed/$(echo ${base} |sed -e 's/.dedup.je.bam/.collapsed.bed/')" 
outbedshifted="Xlsite/shifted/bed/$(echo ${base} |sed -e 's/.dedup.je.bam/.shifted.bed/')" 
bedtools bamtobed -i $file > $outbed 
bedtools shift -m 1 -p -1 -i $outbed -g GENOMEDIR/sizes.genome > $outbedshifted 
echo ""
done

mkdir -p Xlsite/shifted/bedgraph

for file in Xlsite/shifted/bed/*.shifted.bed 
do 
echo "Processing $file" 
base=$(basename $file .shifted.bed) 
outbgplus="Xlsite/shifted/bedgraph/$(echo ${base}).XL.RPM.+.bedgraph" 
outbgminus="Xlsite/shifted/bedgraph/$(echo ${base}).XL.RPM.-.bedgraph" 
TmpScale=$(bc <<< "scale=6;1000000/$(awk -v var="$base" '$1 == var { print $2}' Dedup/Dedup-read-counts.txt)") 
bedtools genomecov -bg -strand + -5 -i $file -g GENOMEDIR/sizes.genome -scale $TmpScale > $outbgplus 
bedtools genomecov -bg -strand - -5 -i $file -g GENOMEDIR/sizes.genome -scale $TmpScale > $outbgminus 
echo ""
done


mkdir -p Xlsite/shifted/bedgraph_sorted

LC_COLLATE=C

for file in Xlsite/shifted/bedgraph/*.bedgraph 
do 
echo "Processing $file" 
base=$(basename $file .bedgraph) 
output="Xlsite/shifted/bedgraph_sorted/$(echo ${base}).sorted.bedgraph" 
sort -k1,1 -k2,2n $file > $output
echo ""
done

# generating error as follows:
#  bedGraphToBigWig in.bedGraph chrom.sizes out.bw
# where in.bedGraph is a four column file in the format:
#       <chrom> <start> <end> <value>
# and chrom.sizes is two column: <chromosome name> <size in bases>
# and out.bw is the output indexed big wig file.
# Use the script: fetchChromSizes to obtain the actual chrom.sizes information
# from UCSC, please do not make up a chrom sizes from your own information.
# The input bedGraph file must be sorted, use the unix sort command:
#   sort -k1,1 -k2,2n unsorted.bedGraph > sorted.bedGraph
mkdir -p Xlsite/shifted/bw

for file in Xlsite/shifted/bedgraph_sorted/*.bedgraph 
do 
echo "Processing $file" 
base=$(basename $file .sorted.bedgraph) 
output="Xlsite/shifted/bw/$(echo ${base}).bw" 
/software/kentUtils-v302.1.0/bin/linux.x86_64/bedGraphToBigWig –switches $file GENOMEDIR/sizes.genome $output
echo ""
done

mkdir -p DEW-seq/Extract

for file in Dedup/*.dedup.je.bam
do
echo "File is $file"
base=$(basename "$file")
output="DEW-seq/Extract/$(echo ${base} |sed -e 's/.dedup.je.bam/.extract.bed/')"
htseq-clip extract \
-i $file \
-e 1 \
-s s \
-g -1 \
--ignore \
-o $output
echo ""
done
 
mkdir -p DEW-seq/Count_w50s20

for file in DEW-seq/Extract/*.bed
do
echo "File is $file"
base=$(basename "$file")
output="DEW-seq/Count_w50s20/$(echo ${base} |sed -e 's/.extract.bed/.count.bed/')"
htseq-clip count \
-i $file \
-a GENOMEDIR/sliding_window/HS.GRCh38.SINV_attribute-fix.flattened.w50s20.txt.gz \
-o $output
echo ""
done


mkdir -p DEW-seq/Matrix_w50s20

htseq-clip createMatrix \
-i DEW-seq/Count_w50s20 \
-e .bed \
-o DEW-seq/Matrix_w50s20/Full_NOP2_iCLIP.matrix.txt.gz




we will proceed to the DEWseq analysis in the second script file named as Downstream analysis.Rmd 